
# PyTorchによる深層学習

> 深層学習パッケージ Pytorch の基本的な使用法を紹介する． 

## 簡単な計算

PyTorchパッケージを用いて $N$ 行 $D$ 列のランダムな行列を作り， 簡単な計算をする． 最後に， 計算された $c = \sum x*y + z$ を $x$ に対する勾配（微分値）をbackwardで計算する． 

randでランダムな行列をを生成する際に， 引数 requires_grad を True に設定しておくことによって， 勾配が計算される．　

変数に対しては，　dataで値が， 勾配を計算するように指定した変数に対しては，gradで勾配が得られる．


In [ ]:
import torch

In [ ]:
N, D = 3,4 #3行4列のランダムなテンソルを生成し， 勾配を計算する
x = torch.rand( (N, D), requires_grad=True)
y = torch.rand( (N, D), requires_grad=True)
z = torch.rand( (N, D), requires_grad=True)

a = x*y
b = a+z
c = torch.sum(b)

c.backward()

c.data

tensor(7.7942)

In [ ]:
print(x.data)

tensor([[0.4322, 0.4083, 0.5727, 0.7827],
        [0.6399, 0.5336, 0.7069, 0.9943],
        [0.9654, 0.9505, 0.1478, 0.4963]])


In [ ]:
print(x.grad)

tensor([[0.5993, 0.3683, 0.9010, 0.6827],
        [0.3640, 0.7956, 0.9216, 0.3772],
        [0.9165, 0.4953, 0.0958, 0.9175]])


##  最小2乗法

$y = a x$ のパラメータ $a$ の最適化を最小2乗法によって行う．

from_numpyでNumPyの配列をPyTorchのテンソルに変換できる．　

パラメータ $a$ をランダムに設定し，勾配を計算するように指示する．

予測値 yhat を計算した後で， 損出関数 loss を誤差の2乗平均として計算し， backward で勾配を計算する．

勾配の逆方向に学習率 lr (learning rate) だけ移動させたものを新しい $a$ とし， それをn_epochs回繰り返す．

ここで，反復ごとに勾配を $0$ に初期化する a.grad.zero_ を呼び出すことに注意されたい．


In [ ]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn

In [ ]:
x_numpy = np.array([1.0, 2.0, 3.0, 4.0, 5.0], dtype=np.float32)
y_numpy = np.array([15.0, 20.0, 34.0, 42.0, 58.0], dtype=np.float32)

x = torch.from_numpy(x_numpy)
y = torch.from_numpy(y_numpy)

a = torch.rand(1, requires_grad=True) #スカラーを定義

n_epochs = 10  #反復回数（エポック数）
lr = 0.01      #学習率
for epoch in range(n_epochs):
    yhat = a*x 
    error = y - yhat
    loss =(error**2).mean()
    
    loss.backward()
    
    with torch.no_grad():
        a -= lr*a.grad

    print(loss.data, a.data)
    
    a.grad.zero_()

tensor(1361.3071) tensor([2.5252])
tensor(830.7396) tensor([4.4297])
tensor(507.9425) tensor([5.9151])
tensor(311.5527) tensor([7.0738])
tensor(192.0692) tensor([7.9776])
tensor(119.3753) tensor([8.6825])
tensor(75.1485) tensor([9.2324])
tensor(48.2408) tensor([9.6612])
tensor(31.8702) tensor([9.9958])
tensor(21.9103) tensor([10.2567])


## 線形回帰


### クラス

nn.Moduleクラスから派生させて線形回帰を行うクラス LinearRegression を作る．

コンストラクタ __init__ で親クラスを呼び出した後でパラメータ $a$ を定義する．

与えられた $x$ に対して予測値 $y=ax$ を計算するための関数 forward を定義する．

モデルのインスタンス model のstate_dicメソッドでパラメータ $a$ の現在の値の辞書を得ることができる． 

In [ ]:
class LinearRegression(nn.Module):
    #コンストラクタ
    def __init__(self):
        super().__init__() #親クラスのコンストラクタを呼ぶ
        self.a = nn.Parameter(torch.rand(1, requires_grad=True)) #モデルのパラメータを準備
        
    #予測値の計算
    def forward(self, x):
        return self.a*x
        
model = LinearRegression()

model.state_dict()  #パラメータと現在の値の辞書

OrderedDict([('a', tensor([0.0902]))])

### 訓練

損出関数を計算する関数 loss_fn を最小2乗誤差 nn.MSELoss とし， 最適化は率的勾配降下法 (SGD: Stochastic Fradient Descent) optim.SDG とする．
引数はモデルのパラメータ model.parameters() と学習率 lr である．

optimizerのstepで勾配降下法の1反復を行い， 反復ごとに zero_grad で勾配を $0$ に初期化する．

In [ ]:
loss_fn = nn.MSELoss()  #損出関数（最小2乗誤差）
optimizer = optim.SGD(model.parameters(), lr)  # 最適化（確率的勾配降下法）を準備； model.parameters()はパラメータを返すジェネレータ

for epoch in range(n_epochs):
    model.train()           #モデルを訓練モードにする
    yhat = model(x)         #forwardメソッドで予測値を計算する
    loss = loss_fn(y, yhat) #損出関数
    loss.backward()         #誤差逆伝播で勾配を計算
    
    optimizer.step()        #最適化の１反復
    optimizer.zero_grad()   #勾配を０にリセット
    
    print(loss.data, model.state_dict())

tensor(15.8396) OrderedDict([('a', tensor([10.4606]))])
tensor(12.1573) OrderedDict([('a', tensor([10.6193]))])
tensor(9.9170) OrderedDict([('a', tensor([10.7431]))])
tensor(8.5540) OrderedDict([('a', tensor([10.8396]))])
tensor(7.7247) OrderedDict([('a', tensor([10.9149]))])
tensor(7.2202) OrderedDict([('a', tensor([10.9736]))])
tensor(6.9132) OrderedDict([('a', tensor([11.0194]))])
tensor(6.7265) OrderedDict([('a', tensor([11.0551]))])
tensor(6.6129) OrderedDict([('a', tensor([11.0830]))])
tensor(6.5438) OrderedDict([('a', tensor([11.1047]))])


## 線形層の追加

nn.Linear(入力数, 出力数)で線形層をモデルに追加する． モデルは $y = w_0 + w_1 x$ となる． 

データの $x,y$ は縦ベクトル（shapeは (5,1)　）になおしておく．

In [ ]:
class LayerLinearRegression(nn.Module):
    #コンストラクタ
    def __init__(self):
        super().__init__() #親クラスのコンストラクタを呼ぶ
        self.linear = nn.Linear(1,1, dtype=torch.float32) #1入力・１出力の線形層
        #self.linear =nn.Sequential(nn.Linear(1,2), nn.ReLU(), nn.Linear(2,1)) #多層のモデルもSequentialを用いて作れる
    #予測値の計算
    def forward(self, x):
        return self.linear(x)
        
model = LayerLinearRegression()

model.state_dict()  #パラメータと現在の値の辞書

OrderedDict([('linear.weight', tensor([[-0.2438]])),
             ('linear.bias', tensor([-0.6258]))])

In [ ]:
x = x.reshape(-1,1)
y = y.reshape(-1,1)

In [ ]:
loss_fn = nn.MSELoss()  #損出関数（最小2乗誤差）
optimizer = optim.SGD(model.parameters(), lr)  # 最適化（確率的勾配降下法）を準備； model.parameters()はパラメータを返すジェネレータ

n_epochs=10
for epoch in range(n_epochs):
    model.train()           #モデルを訓練モードにする
    yhat = model(x)         #forwardメソッドで予測値を計算する
    loss = loss_fn(y, yhat) #損出関数
    loss.backward()         #誤差逆伝播で勾配を計算
    
    optimizer.step()        #最適化の１反復
    optimizer.zero_grad()   #勾配を０にリセット
    
    print(loss.data, model.state_dict())

tensor(1486.0369) OrderedDict([('linear.weight', tensor([[2.3074]])), ('linear.bias', tensor([0.0773]))])
tensor(868.5944) OrderedDict([('linear.weight', tensor([[4.2551]])), ('linear.bias', tensor([0.6133]))])
tensor(508.7790) OrderedDict([('linear.weight', tensor([[5.7422]])), ('linear.bias', tensor([1.0218]))])
tensor(299.0960) OrderedDict([('linear.weight', tensor([[6.8776]])), ('linear.bias', tensor([1.3328]))])
tensor(176.9026) OrderedDict([('linear.weight', tensor([[7.7446]])), ('linear.bias', tensor([1.5695]))])
tensor(105.6939) OrderedDict([('linear.weight', tensor([[8.4066]])), ('linear.bias', tensor([1.7494]))])
tensor(64.1965) OrderedDict([('linear.weight', tensor([[8.9122]])), ('linear.bias', tensor([1.8860]))])
tensor(40.0135) OrderedDict([('linear.weight', tensor([[9.2983]])), ('linear.bias', tensor([1.9896]))])
tensor(25.9204) OrderedDict([('linear.weight', tensor([[9.5933]])), ('linear.bias', tensor([2.0679]))])
tensor(17.7072) OrderedDict([('linear.weight', tensor([[9

## データセットとデータローダー

データセットはTensorDatasetで生成されるサプライチェーン． 

1バッチずつデータを出力するデータローダーは DataLoaderクラスに，データセットを入れることによって生成される．

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
train_data = TensorDataset(x,y)
train_data[0]

(tensor([1.]), tensor([15.]))

In [ ]:
train_loader = DataLoader(dataset = train_data, batch_size=2, shuffle=True)
for x_batch, y_batch in train_loader:
    print(x_batch, y_batch)

tensor([[3.],
        [2.]]) tensor([[34.],
        [20.]])
tensor([[4.],
        [1.]]) tensor([[42.],
        [15.]])
tensor([[5.]]) tensor([[58.]])


In [ ]:
loss_fn = nn.MSELoss()  #損出関数（最小2乗誤差）
optimizer = optim.SGD(model.parameters(), lr)  # 最適化（確率的勾配降下法）を準備； model.parameters()はパラメータを返すジェネレータ

n_epochs=10
for epoch in range(n_epochs):
    for x_batch, y_batch in train_loader:
        model.train()           #モデルを訓練モードにする
        yhat = model(x_batch)         #forwardメソッドで予測値を計算する
        loss = loss_fn(y_batch, yhat) #損出関数
        loss.backward()         #誤差逆伝播で勾配を計算

        optimizer.step()        #最適化の１反復
        optimizer.zero_grad()   #勾配を０にリセット

        print(loss.data, model.state_dict())

tensor(4.4772) OrderedDict([('linear.weight', tensor([[9.8559]])), ('linear.bias', tensor([2.1335]))])
tensor(26.2251) OrderedDict([('linear.weight', tensor([[10.2154]])), ('linear.bias', tensor([2.2294]))])
tensor(1.1903) OrderedDict([('linear.weight', tensor([[10.1281]])), ('linear.bias', tensor([2.2076]))])
tensor(4.5405) OrderedDict([('linear.weight', tensor([[10.1970]])), ('linear.bias', tensor([2.2483]))])
tensor(4.0279) OrderedDict([('linear.weight', tensor([[10.1027]])), ('linear.bias', tensor([2.2115]))])
tensor(27.8253) OrderedDict([('linear.weight', tensor([[10.6302]])), ('linear.bias', tensor([2.3170]))])
tensor(9.6045) OrderedDict([('linear.weight', tensor([[10.6852]])), ('linear.bias', tensor([2.3066]))])
tensor(2.0820) OrderedDict([('linear.weight', tensor([[10.6945]])), ('linear.bias', tensor([2.3230]))])
tensor(9.6154) OrderedDict([('linear.weight', tensor([[10.4464]])), ('linear.bias', tensor([2.2610]))])
tensor(5.0532) OrderedDict([('linear.weight', tensor([[10.3953]